In [1]:
! pip install polars


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl

In [3]:
def read_polars_df(filename):
    # Step 1: Read just the header
    with open(filename, "r") as f:
        headers = f.readline().strip().split(",")
    
    # Step 2: Build schema with all columns as Utf8
    schema = {col: pl.Utf8 for col in headers}
    
    # Step 3: Read CSV with schema
    df = pl.read_csv(filename, schema_overrides=schema)
    
    # Step 4: Drop duplicate rows
    df = df.unique()
    
    return df

def calculate_unique(df: pl.DataFrame, release) -> int:
    # Drop "_version_" column if it exists
    print(f"Number of rows in initial dataset DR{release}: {df.height}")
    if "_version_" in df.columns:
        df = df.drop("_version_")
    
    # Remove duplicated rows
    df = df.unique()
    
    # Return number of rows
    print(f"Number of rows without duplications DR{release}: {df.height}\n")
    return df

def count_procedures(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df.group_by("procedure_stable_id")
          .agg(pl.len().alias("count"))
          .sort("count", descending=True)
    )

def compare_procedure_counts(procedures_23: pl.DataFrame, procedures_22: pl.DataFrame) -> pl.DataFrame:
    return (
        procedures_23.rename({"count": "dr_23"})
        .join(procedures_22.rename({"count": "dr_22"}), on="procedure_stable_id", how="full")
        .with_columns([
            pl.col("dr_23").cast(pl.Int32).fill_null(0),
            pl.col("dr_22").cast(pl.Int32).fill_null(0),
        ])
        .with_columns(
            (pl.col("dr_23") - pl.col("dr_22")).alias("diff")
        )
        .select(["procedure_stable_id", "dr_22", "dr_23", "diff"])
        .sort("diff", descending=True)
    )

In [4]:
df_22 = read_polars_df("statistical-results-ALL-22.1.csv")
df_23 = read_polars_df("statistical-results-ALL-23.0.csv")

In [5]:
df_22_unique = calculate_unique(df_22, "22.1")
df_23_unique = calculate_unique(df_23, "23.0")

Number of rows in initial dataset DR22.1: 3165323
Number of rows without duplications DR22.1: 2105158

Number of rows in initial dataset DR23.0: 2159930
Number of rows without duplications DR23.0: 2159930



In [6]:
procedures_22 = count_procedures(df_22)
procedures_23 = count_procedures(df_23)
comparison = compare_procedure_counts(procedures_23, procedures_22)
comparison.write_csv("procedure_comparison.csv")

compare_procedure_counts(procedures_23, procedures_22)

procedure_stable_id,dr_22,dr_23,diff
str,i32,i32,i32
"""IMPC_CSD_003""",195724,205262,9538
"""IMPC_EYE_003""",75478,81146,5668
"""IMPC_EMA_002""",324,4070,3746
"""IMPC_CBC_003""",83789,87192,3403
"""IMPC_HEM_002""",100242,103129,2887
…,…,…,…
"""BCMLA_PAT_002""",17635,1503,-16132
"""MGP_PBI_001""",40971,21625,-19346
"""UCDLA_PAT_002""",24049,3623,-20426
